In [31]:
import pandas as pd
import numpy as np
import csv
import os
import random
import re
import requests
import xlrd
import matplotlib.pyplot as plt
import math
import datetime

In [1]:
# use 000300 as benchmark;399906 as fund
index = pd.read_csv('ind_return.csv')
indus = pd.read_csv('stock_ind.csv')
s300 = pd.read_csv('index_300.csv')
s800 = pd.read_csv('index_800.csv')

In [68]:
# merge industry code with each stock in 000300
indus['Stkcd1'] = indus['Stkcd1'].astype('str')
s300['Stkcd'] = s300['Stkcd'].astype('str')

Stkcd = []
for i in range(len(indus['Stkcd1'])):
    Stkcd.append(str(indus['Stkcd1'][i])[0:6])
Stkcd   
indus['Stkcd'] = Stkcd
indus['Stkcd'] = indus['Stkcd'].astype('int')
indus['Stkcd'] = indus['Stkcd'].astype('str')
ind_l = indus[['Stkcd','行业指数代码']]

stand = pd.merge(s300,ind_l,on = 'Stkcd',how = 'left')
stand.to_csv('index300_ind.csv',encoding = 'utf-8-sig')

# caculate weight of industry in benchmark
gp = stand.groupby(['行业指数代码','Enddt'])['Weight'].sum()
gp = pd.DataFrame(gp)
gp = gp.reset_index()
gp.to_csv('300_ind.csv',encoding = 'utf-8-sig')

In [ ]:
# ---择时收益：持仓比重*行业指数---

gp=ind300
ind300 = ind300[(ind300.行业指数代码 != '0')]

ind_r = index.values
from itertools import chain
find_r = (list(chain.from_iterable(ind_r)))
ind300['ind_r'] = find_r

ind300['ind_r_total'] = ind300['Weight']*ind300['ind_r']/100
zs = ind300.groupby(['Enddt'])['ind_r_total'].sum()
zs = pd.DataFrame(zs)
zs = zs.reset_index()
zs.to_csv('zs.csv',encoding = 'utf-8-sig')

In [4]:
# merge industry code with each stock in 399906
s800['Stkcd'] = s800['Stkcd'].astype('str')
s800 = pd.merge(s800,ind_l,on = 'Stkcd',how = 'left')

gp = stand.groupby(['行业指数代码','Enddt'])['Weight'].sum()
gp = pd.DataFrame(gp)
gp = gp.reset_index()
gp.to_csv('800_ind.csv',encoding = 'utf-8-sig')
gp=stand

#daily return of each stock
dyr = pd.read_csv('daily_return.csv')

dyr['Stkcd'] = dyr['Stkcd'].astype('int')
dyr['Stkcd'] = dyr['Stkcd'].astype('str')
dyr.rename(columns={'Trddt':'Enddt'}, inplace=True) 

stk800 = pd.merge(stand,dyr,on = ['Stkcd','Enddt'],how = 'left')
stk800.dropna(inplace = True)
print(stk300.head())

stk800['dyr'] = stk800['Weight']*stk800['Dretwd']
gp = stk800.groupby(['Enddt','行业指数代码'])['dyr'].sum()
gp = pd.DataFrame(gp)
gp = gp.reset_index()
gp = gp[(gp.行业指数代码!= '0')]
print(gp.head())
gp.to_csv('800_ind_r.csv',encoding = 'utf-8-sig')

In [6]:
# ---选股收益：行业比重*行业持仓收益---
ind800 = ind800[(ind800.行业指数代码!= '0')]
box_300r = pd.merge(ind800,gp,on = ['行业指数代码','Enddt'],how = 'left')
box_300r['ind_r'] = box_300r['Weight']*box_300r['dyr']
box_300r.rename(columns={'Weight':'基准行业比重','dyr':'持仓行业回报率加权','ind_r':'行业基准比重*行业持仓收益'}, inplace=True) 
box_300r['行业基准比重*行业持仓收益'] = box_300r['行业基准比重*行业持仓收益']*100

xg = box_300r.groupby(['Enddt'])['ind_r'].sum()
xg = pd.DataFrame(xg)
xg = xg.reset_index()
xg.head()
xg.to_csv('xg.csv',encoding = 'utf-8-sig')

In [5]:
res = pd.read_excel('指数回报.xlsx',encoding = 'gbk')
res.rename(columns={'Idxtrd01':'Enddt','Idxtrd08':'dyr'}, inplace=True) 

r300 = res[(res.Indexcd == '000300')]
r300 = r300[['Enddt', 'dyr']]
r800 = res[(res.Indexcd == '000906')]
r800 = r800[['Enddt', 'dyr']]

tbl = pd.merge(zs,xg,on = 'Enddt',how='left')
tbl.rename(columns={'ind_r_total':'行业偏离','ind_r':'个股选择'}, inplace=True) 
tbl = pd.merge(tbl,r300,on = 'Enddt',how='left' )
tbl = pd.merge(tbl,r800,on = 'Enddt',how='left' )
tbl.rename(columns={'dyr_x':'超额收益率','dyr_y':'基准'}, inplace=True) 
tbl.dropna(inplace=True)
tbl['其他'] = tbl['超额收益率']-tbl['个股选择']-tbl['行业偏离']+tbl['基准']
tbl['择时收益'] = tbl['行业偏离'] - tbl['基准']
tbl['选股收益'] = tbl['个股选择'] - tbl['基准']
ktbl = tbl[['交易日期', '超额收益率',  '择时收益', '选股收益', '其他', '基准']]
print(ktbl.head())
ktbl.to_csv('final_brinson.csv',encoding = 'utf-8-sig')